
# 🎮 **Create a Minecraft Server on Google Colab!**
🚀 Set up and run your own **Minecraft server** effortlessly!

---

## 📌 **How It Works:**  
✔️ **The script below will initiate your server.**  
✔️ **First, you'll need to create the server** before running it.  
✔️ **No worries!** The scripts below will handle most of the setup for you.  

💡 **Want to change the server's region?** Check the instructions below.

---

## 🔗 **Original Repository:**  
[GitHub - agrawalchaitany/minecolab](https://github.com/agrawalchaitany/minecolab.git)

## ⚡ **Modified Version:**  
✅ **Well-Optimized for Performance**  

---



```
███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗
████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝
██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝

```

🎉 **Enjoy your Minecraft server on Google Colab!**


# 🛠️ **Step 1: Create the Server (First-Time Setup Only)**
🚀 **Run this step only once** to set up your server and accept the EULA.

---

## 📌 **What This Does:**  
✔️ Creates your **Minecraft server**  
✔️ Automatically **accepts the EULA**  
✔️ Prepares the server for **quick startup**  

---

🎮 **After this, your server will be ready to launch!** 🚀


# ⬇️ **Download the Minecraft Server**  
🚀 **Get a high-performance Paper server with plugin support!**  

---

## 📌 **About This Script**  
✔️ **Downloads** **Paper**, a high-performance Vanilla server.  
✔️ **Supports plugins** for an enhanced gameplay experience.  
✔️ **Want a different `server.jar`?** Manually place it in your Drive folder.  

📂 **Storage Location:**  
The server files will be **saved on your Google Drive**  
(Linked to the Google account you choose or are currently using).  

---

🎮 **Customize & enjoy your Minecraft server!** 🚀


In [ ]:
################################################################################
#                            Available Minecraft Versions                        #
################################################################################

# Paper Server Type:
# -------------------
# Choose the desired version from the following tested versions:
# - 1.21.4
# - 1.21.3
# - 1.21.1
# - 1.21
# - 1.20.6
# - 1.20.4
# - 1.20.3
# - 1.20.2
# - 1.20.1
# - 1.20
# - 1.19.4
# - 1.19.3
# - 1.19.2
# - 1.19.1
# - 1.19
# - 1.18.2
# - 1.18.1
# - 1.18
# - 1.17.1
# - 1.17
# - 1.16.5
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8

# Example:
# MINECRAFT_VERSION = "1.21.4"

# Note: Newer versions might work but are not guaranteed to be fully supported.

################################################################################

# Fabric Server Type:
# --------------------
# Choose from the following tested versions:
# - "25w07a", "25w06a", "25w05a", "25w04a", "25w03a", "25w02a"
# - "1.21.4", "1.21.4-rc3", "1.21.4-rc2", "1.21.4-rc1", "1.21.4-pre3", "1.21.4-pre2", "1.21.4-pre1"
# - "1.21.3", "1.21.2", "1.21.2-rc2", "1.21.2-rc1", "1.21.2-pre5", "1.21.2-pre4", "1.21.2-pre3", "1.21.2-pre2", "1.21.2-pre1"
# - "1.21.1", "1.21.1-rc1", "1.21", "1.21-rc1", "1.21-pre4", "1.21-pre3", "1.21-pre2", "1.21-pre1"
# - "1.20.6", "1.20.6-rc1", "1.20.5", "1.20.5-rc3", "1.20.5-rc2", "1.20.5-rc1", "1.20.5-pre4", "1.20.5-pre3", "1.20.5-pre2", "1.20.5-pre1"
# - "1.20.4", "1.20.3", "1.20.3-rc1", "1.20.3-pre4", "1.20.3-pre3", "1.20.3-pre2", "1.20.3-pre1"
# - "1.20.2", "1.20.2-rc2", "1.20.2-rc1", "1.20.2-pre4", "1.20.2-pre3", "1.20.2-pre2", "1.20.2-pre1"
# - "1.20.1", "1.20.1-rc1", "1.20", "1.20-rc1", "1.20-pre7", "1.20-pre6", "1.20-pre5", "1.20-pre4", "1.20-pre3", "1.20-pre2", "1.20-pre1"
# - "1.19.4", "1.19.4-rc3", "1.19.4-rc2", "1.19.4-rc1"

# Example:
# MINECRAFT_VERSION = "25w07a"

################################################################################

# Available Versions for Fabric Loader:
# --------------------------------------
# Select the appropriate version from the following list:
# - '0.16.10', '0.16.9', '0.16.8', '0.16.7', '0.16.6', '0.16.5', '0.16.4', 
# - '0.16.3', '0.16.2', '0.16.1', '0.16.0', '0.15.11', '0.15.10', '0.15.9', 
# - '0.15.8', '0.15.7', '0.15.6', '0.15.5', '0.15.4', '0.15.3', '0.15.2', 
# - '0.15.1', '0.15.0', '0.14.25', '0.14.24', '0.14.23', '0.14.22', '0.14.21', 
# - '0.14.20', '0.14.19', '0.14.18', '0.14.17', '0.14.16', '0.14.15', '0.14.14', 
# - '0.14.13', '0.14.12', '0.14.11', '0.14.10', '0.14.9', '0.14.8', '0.14.7', 
# - '0.14.6', '0.14.5', '0.14.4', '0.14.3', '0.14.2', '0.14.1', '0.14.0', 
# - '0.13.3', '0.13.2', '0.13.1', '0.13.0', '0.12.12', '0.12.11', '0.12.10', 
# - '0.12.9', '0.12.8', '0.12.7', '0.12.6', '0.12.5', '0.12.4', '0.12.3', 
# - '0.12.2', '0.12.1', '0.12.0'

# Example:
# Fabric_Loader_Version = "0.16.10"

################################################################################

# Available Installer Versions:
# ------------------------------
# Choose the desired version:
# - '1.0.1', '1.0.0', '0.11.2', '0.11.1', '0.11.0', '0.10.2', '0.10.1', 
# - '0.10.0', '0.9.1', '0.9.0', '0.8.3', '0.8.2', '0.8.1', '0.8.0'

# Example:
# Installer_Version = "1.0.1"

################################################################################

# Choose Server Type:
# -------------------
# Currently Available Options:
# - paper (Most vanilla. Same as SPIGOT, bukkit, etc.)
# - forge (For using MODS. Installation takes ~10-15 minutes. Wait for "COMPLETED" after the first step.)
# - fabric (For playing Fabric, an alternative to Forge)
# - purpur (A highly customizable and enhanced Paper server)

# Note: Forge is not available due to API issues.
################################################################################


import requests
import json
import os
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive")

# Configuration
MINECRAFT_VERSION = "1.20.2"
SERVER_TYPE = "fabric"  # Options: "paper", "fabric", "purpur", "forge"
Fabric_Loader_Version = "0.16.10" # For fabric server only
Installer_Version = "1.0.1" # For fabric server only
SERVER_DIR = "/content/drive/My Drive/Minecraft-server"
os.makedirs(SERVER_DIR, exist_ok=True)

# Function to fetch the latest PaperMC build
def get_latest_paper_build(version):
    url = f"https://api.papermc.io/v2/projects/paper/versions/{version}/builds"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        builds = [build["build"] for build in data.get("builds", []) if build.get("channel") == "default"]
        return builds[-1] if builds else None
    else:
        print("Failed to fetch PaperMC API.")
        return None

# Fetch latest builds for different server types
latest_build = get_latest_paper_build(MINECRAFT_VERSION) if SERVER_TYPE == "paper" else None

# Generate download URLs
server_urls = {
    "paper": f"https://api.papermc.io/v2/projects/paper/versions/{MINECRAFT_VERSION}/builds/{latest_build}/downloads/paper-{MINECRAFT_VERSION}-{latest_build}.jar" if latest_build else None,
    "forge": f"https://serverjars.com/api/fetchJar/modded/forge/{MINECRAFT_VERSION}",
    "fabric": f"https://meta.fabricmc.net/v2/versions/loader/{MINECRAFT_VERSION}/{Fabric_Loader_Version}/{Installer_Version}/server/jar",
    "purpur": None
}

# Fetch latest Purpur build if needed
if SERVER_TYPE == "purpur":
    purpur_response = requests.get(f"https://api.purpurmc.org/v2/purpur/{MINECRAFT_VERSION}")
    if purpur_response.status_code == 200:
        purpur_data = purpur_response.json()
        latest_purpur_build = purpur_data.get("builds", {}).get("latest")
        if latest_purpur_build:
            server_urls["purpur"] = f"https://api.purpurmc.org/v2/purpur/{MINECRAFT_VERSION}/{latest_purpur_build}/download"

# Define jar filenames
jar_name = {
    "paper": "server.jar",
    "fabric": "fabric-installer.jar",
    "forge": "forge-installer.jar",
    "purpur": "purpur.jar"
}

download_url = server_urls.get(SERVER_TYPE)

# Download the selected server
if download_url:
    print(f"Downloading {SERVER_TYPE} server from {download_url}...")
    response = requests.get(download_url)
    
    if response.status_code == 200:
        with open(os.path.join(SERVER_DIR, jar_name[SERVER_TYPE]), "wb") as f:
            f.write(response.content)
        print(f"{SERVER_TYPE} server downloaded successfully.")
    else:
        print(f"Failed to download {SERVER_TYPE} server. Status code: {response.status_code}")
else:
    print(f"Invalid server type or missing URL: {SERVER_TYPE}")

# Run server installation if necessary
if SERVER_TYPE == "fabric":
    os.system(f"java -jar {os.path.join(SERVER_DIR, 'fabric-installer.jar')} server -mcversion {MINECRAFT_VERSION} -downloadMinecraft")

if SERVER_TYPE == "forge":
    os.chdir(SERVER_DIR)
    os.system(f"java -jar {jar_name['forge']} --installServer")

# Save configuration
config_data = {"server_type": SERVER_TYPE, "server_version": MINECRAFT_VERSION }
if SERVER_TYPE == 'fabric':
    config_data['Fabric_loader_version'] = Fabric_Loader_Version
    config_data['installer_version'] = Installer_Version
with open(os.path.join(SERVER_DIR, "colabconfig.json"), "w") as config_file:
    json.dump(config_data, config_file)

print("Setup complete!")

# Accept Minecraft EULA
eula_path = os.path.join(SERVER_DIR, "eula.txt")
with open(eula_path, "w") as eula_file:
    eula_file.write("eula=true\n")

print("EULA accepted. Ready to start the server!")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server
Completed!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server


# 🚀 **Start the Minecraft Server**
💡 **Use this whenever you want to open it.**

---

In [ ]:
import os
import time
import json
import re
from google.colab import drive

# 🛠️ Update package lists
print("\n🔄 Updating package lists...")
os.system("sudo apt update &>/dev/null && echo '✅ apt cache successfully updated' || echo '❌ apt cache update failed.'")

# 📂 Mount Google Drive
print("\n🔗 Mounting Google Drive...")
drive.mount('/content/drive')

# 🎮 Change directory to Minecraft server folder
server_path = "/content/drive/My Drive/Minecraft-server"
if os.path.exists(server_path):
    os.chdir(server_path)
    print(f"✅ Changed working directory to: {server_path}")
    print("📂 Listing server files:\n")
    !ls
else:
    print(f"❌ Error: {server_path} not found. Make sure the folder exists in Google Drive.")
    exit()

# 🔧 Load or create configuration file
config_file = "colabconfig.json"
if os.path.isfile(config_file):
    with open(config_file, "r") as f:
        colabconfig = json.load(f)
else:
    colabconfig = {"server_type": "generic"}
    with open(config_file, "w") as f:
        json.dump(colabconfig, f)

server_type = colabconfig.get("server_type", "generic")

# ☕ Install OpenJDK 21
print("\n🔧 Installing OpenJDK 21...")
os.system("sudo apt-get install openjdk-21-jre-headless -y &>/dev/null")

# ✅ Verify Java installation
java_version = os.popen("java -version 2>&1 | awk -F '\"' 'NR==1 {print $2}'").read().strip()
if java_version.startswith("21"):
    print(f"✅ OpenJDK {java_version} is working correctly.")
else:
    print(f"⚠️ OpenJDK 21 is not installed or not working. Detected Java version: {java_version}")
    exit()

# 🗂️ Server jar names
jar_list = {
    'paper': 'server.jar',
    'fabric': 'fabric-installer.jar',
    'generic': 'server.jar',
    'forge': 'forge.jar',
    'purpur': 'purpur.jar'
}
jar_name = jar_list.get(server_type, "server.jar")

# 📥 Download Purpur server jar if needed
if server_type == "purpur" and not os.path.isfile("purpur.jar"):
    print("\n📥 Downloading Purpur server jar...")
    os.system("curl -o purpur.jar https://api.purpurmc.org/v2/purpur/1.21/latest/download")

# 🔧 Java arguments for better performance
if server_type in ["paper", "purpur"]:
    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
    server_flags = ""

memory_allocation = "-Xms8704M -Xmx8704M"

# 🌍 tunneling service availble
#  -nglocalhost
#  -pinggy

# 🌍 Setup tunneling service
tunnel_service = "nglocalhost"  # Can be changed to "pinggy" if needed
print(f"\n🌍 Using {tunnel_service} for server tunneling...")
key_path = "/content/drive/My Drive/Minecraft-server/ssh_keys/id_rsa"

# ⏳ Generate SSH key if missing.
def generate_ssh_key(key_path):
    
    if not os.path.isfile(key_path):
        os.makedirs(os.path.dirname(key_path), exist_ok=True)
        os.system(f"ssh-keygen -t rsa -b 4096 -f '{key_path}' -N '' -q")
        os.system(f"chmod 400 '{key_path}'")
        os.system(f"ssh-keygen -lf '{key_path}.pub' > '{key_path}_fingerprint.txt'")

print(f"\n🔌 Setting up {tunnel_service} tunneling...")

# 🔌 Establishing the tunnel
if tunnel_service == "nglocalhost":    
    
    # ⏳ Generate SSH key if missing
    generate_ssh_key(key_path)

    # Establish tunnel
    print("⏳ Establishing tunnel, please wait...")
    os.system("nohup ssh -T -o StrictHostKeyChecking=no -i '/content/drive/My Drive/Minecraft-server/ssh_keys/id_rsa' -R minecraft03127:25565:localhost:25565 nglocalhost.com > tunnel_log.txt 2>&1 &")
    url_pattern=r"(nglocalhost:[^\s]+)"
    # Wait for tunnel to establish
    time.sleep(10)

elif tunnel_service == "pinggy":    

    # ⏳ Generate SSH key if missing
    generate_ssh_key(key_path)

    # Establish tunnel
    print("⏳ Establishing tunnel, please wait...")
    os.system("nohup ssh -p 443 -o StrictHostKeyChecking=no -o ServerAliveInterval=30 -R0:127.0.0.1:25565 tcp@ap.a.pinggy.io > tunnel_log.txt 2>&1 &")
    url_pattern=r"(tcp://[^\s]+)"
    # Wait for tunnel to establish
    time.sleep(10)

else:
    print("⚠️ No tunneling service selected. Server will run locally.")

# Extract and display the tunnel URL
with open("tunnel_log.txt", "r") as log_file:
    log_content = log_file.read()
    match = re.search(url_pattern, log_content)  # Extract first URL
    if match:
        tunnel_url = match.group(0)
        print(f"✅ Tunnel established! Access your Minecraft server at:\n🌍 {tunnel_url}")
    else:
        print("⚠️ Could not extract tunnel URL. Check 'tunnel_log.txt' manually.")

# 🚀 Start Minecraft server
print("\n🚀 Starting Minecraft server...")
!java {memory_allocation} {server_flags} -jar {jar_name} nogui
print("✅ Minecraft server closed!")


🔄 Updating package lists...

🔗 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Changed working directory to: /content/drive/My Drive/Minecraft-server
📂 Listing server files:


🔧 Installing OpenJDK 21...
✅ OpenJDK 21.0.5 is working correctly.

🌍 Using nglocalhost for server tunneling...

🔌 Setting up nglocalhost tunneling...
⏳ Establishing tunnel, please wait...
✅ Tunnel established! Access your Minecraft server at:
🌍 nglocalhost.com:25565

🚀 Starting Minecraft server...
Starting net.minecraft.server.Main
[04:41:13] [ServerMain/INFO]: Environment: Environment[sessionHost=https://sessionserver.mojang.com, servicesHost=https://api.minecraftservices.com, name=PROD]
[04:41:17] [ServerMain/INFO]: Loaded 1370 recipes
[04:41:17] [ServerMain/INFO]: Loaded 1481 advancements
[04:41:17] [Server thread/INFO]: Starting minecraft server version 1.21.4
[04:41:17] [Server thread/INFO]: Loading p